In [69]:
import pandas as pd
import re
import statistics
import numpy as np

In [2]:
filename = "C:\\Users\\Peter\\Desktop\\Thinkful\\Wellcome\\WELLCOME_APCspend2013_forThinkful.csv"

In [3]:
df = pd.read_csv(filename, low_memory = False, encoding = 'iso-8859-1')

In [4]:
df.shape

(2127, 5)

In [5]:
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [6]:
precheck = df.groupby('Journal title').count()
precheck = precheck.sort_values(['Article title'], ascending = False)
precheck.head()

,PMID/PMCID,Publisher,Article title,COST (£) charged to Wellcome (inc VAT when charged)
Journal title,,,,
PLoS One,91,92,92,92
PLoS ONE,62,62,62,62
Journal of Biological Chemistry,47,48,48,48
Nucleic Acids Research,20,21,21,21
Proceedings of the National Academy of Sciences,19,19,19,19


In [48]:


source = df[['Journal title', 'COST (£) charged to Wellcome (inc VAT when charged)']]

#simplfy the column headers
source = source.rename(columns = {'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'})
source = source.rename(columns = {'Journal title':'Journal'})

#make a new column to store the simplified journal names
source['Parsed'] = source['Journal']

#remove common text that could cause duplicates
source['Parsed'] = source['Parsed'].str.strip()
source['Parsed'] = source['Parsed'].str.lower()
source['Parsed'] = source['Parsed'].str.replace(':','')
source['Parsed'] = source['Parsed'].str.replace('&','and')
source['Parsed'] = source['Parsed'].str.replace(',','')
source['Cost'] = source['Cost'].str.strip('$')
source['Cost'] = source['Cost'].str.strip('£').astype(float)
source = source[source.Cost < 4000.00]

#correct spelling errors
source['Parsed'] = source['Parsed'].str.replace('antimicobial','antimicrobial')
source['Parsed'] = source['Parsed'].str.replace('agfents','agents')
source['Parsed'] = source['Parsed'].str.replace('angewande','angewandte')
source['Parsed'] = source['Parsed'].str.replace('behaviour','behavior')  
source['Parsed'] = source['Parsed'].str.replace('ophthalmology','opthalmology')  
source['Parsed'] = source['Parsed'].str.replace('jnl','journal') 
source['Parsed'] = source['Parsed'].str.replace('of','') 
source['Parsed'] = source['Parsed'].str.replace('.','') 
source['Parsed'] = source['Parsed'].str.replace('neuropathol','neuropathologica') 
source['Parsed'] = source['Parsed'].str.replace('neuropathologicaogica','neuropathologica') 
source['Parsed'] = source['Parsed'].str.replace('biinformatics','bionformatics') 
source['Parsed'] = source['Parsed'].str.replace('journals','journal') 
source['Parsed'] = source['Parsed'].str.replace('eur j immunol','european journal immunology') 
source['Parsed'] = source['Parsed'].str.replace('european child and adolescent psychiatty','european child and adolescent psychiatry') 
source['Parsed'] = source['Parsed'].str.replace('genetic epidemology','genetic epidemiology') 
source['Parsed'] = source['Parsed'].str.replace('haematologica/the haematology journal','haematologica') 
source['Parsed'] = source['Parsed'].str.replace('heptology','hepatology') 

#remove text that references subsections of journals
source['Parsed'] = source['Parsed'].str.replace('embo journal','embo') 
source['Parsed'] = source['Parsed'].str.replace('embo reports','embo') 
source['Parsed'] = source['Parsed'].str.replace('embo molecular medicine','embo') 

source['Parsed'] = source['Parsed'].str.partition('part')
source['Parsed'] = source['Parsed'].str.partition('section')
source['Parsed'] = source['Parsed'].str.partition('online')

print("There are {} unique Journals in source after parsing".format(source['Parsed'].nunique()))

There are 840 unique Journals in source after parsing


In [67]:
source.head()

,Journal,Cost,Parsed
0,Psychological Medicine,0.00,psychological medicine
1,Biomacromolecules,2381.04,biomacromolecules
2,J Med Chem,642.56,j med chem
3,J Med Chem,669.64,j med chem
4,J Org Chem,685.88,j org chem


In [49]:
testingphrase = 'h'
testing = pd.DataFrame()
testing['Title'] = source['Parsed'].unique()
testing = testing.sort_values(['Title'], ascending=[True])
#testing.loc[(testing['Title'].astype(str).str.startswith(testingphrase))]

In [50]:
checktopjournals = source.groupby('Parsed').count()
checktopjournals = checktopjournals.sort_values(['Journal'], ascending = False)
top5 = checktopjournals.head().index.tolist()

In [51]:
def findmodes (data):
    (values, counts) = np.unique(data, return_counts=True)
    ind = np.argmax(counts)
    modes=[values[ind]]
    countofmode = counts[ind]
    counts = np.delete(counts, ind)
    values = np.delete(values, ind)
    ind = np.argmax(counts)
    if countofmode == counts[ind]:
        modes.append(values[ind])
    return modes

In [62]:
def findvalues(data, dataname):
    print('The MODE of the cost of a', dataname,' article are', ' and '.join(str(e) for e in (findmodes(data))))
    print('The MEAN of the cost of a', dataname,' article is',data.mean())
    print('The STANDARD DEVIATION of the cost of a', dataname, ' article is', data.std(ddof=1))

In [63]:
def runentries (listtorun):
    for e in listtorun:
        current_j = source[source['Parsed']==e]
        print('Here is the information requested for', e.upper(), 'with ', len(current_j.index), 'articles')
        findvalues(current_j['Cost'], e)
        print('')

In [64]:
runentries(top5)

Here is the information requested for PLOS ONE with  182 articles
The MODE of the cost of a plos one  article are 825.68
The MEAN of the cost of a plos one  article is 934.655824176
The STANDARD DEVIATION of the cost of a plos one  article is 201.847588733

Here is the information requested for JOURNAL  BIOLOGICAL CHEMISTRY with  52 articles
The MODE of the cost of a journal  biological chemistry  article are 1276.08 and 1556.61
The MEAN of the cost of a journal  biological chemistry  article is 1423.58846154
The STANDARD DEVIATION of the cost of a journal  biological chemistry  article is 411.954359938

Here is the information requested for NEUROIMAGE with  29 articles
The MODE of the cost of a neuroimage  article are 1762.69 and 2503.34
The MEAN of the cost of a neuroimage  article is 2215.16827586
The STANDARD DEVIATION of the cost of a neuroimage  article is 266.653946919

Here is the information requested for NUCLEIC ACIDS RESEARCH with  26 articles
The MODE of the cost of a nucle